# ASSIGNMENT 6
## Vishanth Hari Raj (206310279)

In [1]:
import numpy as np
import pandas as pd
import random
from scipy.stats import beta
from scipy.stats import t

**Question 1:** Like in class, we want to make an assessment, for each campaign, of its true click-through-rate (abbreviated as "c") and its true average post-click transaction profit volume (abbreviated as "v"). You can think of "v" as the average profit from a click or conversion. We want to make an assessment, for each campaign,  also of its true expected volume per exposure or impression (abbreviated as "EVI"). The true EVI of a campaign is its "c" multiplied by its "v".  For each campaign, compute the following:

1. The bayesian posterior probability that the campaign's true click-through-rate is the highest across all campaigns. Assume the prior is the uniform distribution. To generate draws from the beta distribution, in R use the rbeta() function and in Python use scipy.stats.beta.rvs(a=alpha value, b=beta value, size=number of draws needed). Here alpha value and beta value are, respectively, what we referred to in the sessions as the success parameter and the failure parameter.

2. The bayesian posterior probability that the campaign's true average post-click volume per click is the highest  across all campaigns.To generate draws from the t distribution in R  use the rt() function. In Python, first generate draws from the standardized t distribution using scipy.stats.t.rvs(df= the appropriate degrees of freedom,  size=number of draws needed).  Once you have the draws in either R or Python you have to multiply by the standard error and then add the sample mean as shown in the example Excel  computation to obtain the draws from the target posterior probability density.

3. The bayesian posterior probability that the campaign's true expected volume per exposure (impression) is the highest. 

In the probability computations, use a simulation with 100000 draws (this is higher than in the in-class Excel example which has 5000 draws).

In [2]:
click_df = pd.read_excel('clicks.dataset.2.xlsx', index_col= 'ad')
vol_df = pd.read_excel('volumes.dataset.2.xlsx')

In [3]:
click_df = click_df.T

In [4]:
click_df['s']= click_df['clicks'] +1
click_df['f']= click_df['exposures'] - click_df['clicks'] + 1

In [5]:
click_df

ad  clicks  exposures   s    f
1       52       1000  53  949
2       38       1000  39  963
3       51       1000  52  950
4       45       1000  46  956
5       25       1000  26  976

In [6]:
n_sim = np.zeros((100000,5))

In [7]:
for i in range(len(click_df)):
    samples = beta.rvs(a=click_df.iloc[i]['s'], b=click_df.iloc[i]['f'], size=100000)
    n_sim[:, i] = samples
sims=pd.DataFrame(n_sim)
sims['max_ind']=sims.apply(np.argmax,axis=1)

In [8]:
sims['max_ind'] = sims['max_ind']+1
probs= sims['max_ind'].value_counts()/len(n_sim)

In [9]:
probs=probs.reset_index()
probs.sort_index()
probs.rename(columns={"index": "Ad", "max_ind":"Ad", "count":"Probability"})

Ad  Probability
0   1      0.46882
1   3      0.39835
2   4      0.11646
3   2      0.01635
4   5      0.00002

In [10]:
vol_summ = vol_df[['ad','volume']].groupby('ad').agg(['mean','sem','count'])
vol = vol_summ.reset_index()

In [11]:
vol.columns = ['Ad','Mean','Std Dev','DOF']
vol['DOF'] = vol['DOF'] + 1
vol

Ad       Mean   Std Dev  DOF
0   1  42.076923  1.391324   53
1   2  57.289474  2.822110   39
2   3  41.803922  1.945134   52
3   4  47.888889  2.017655   46
4   5  90.640000  6.137285   26

In [12]:
n_sim2 = np.zeros((100000,5))

In [13]:
for i in range(len(vol)):
    se = vol.loc[i,'Std Dev']
    sm = vol.loc[i,'Mean']
    dof = vol.loc[i,'DOF']
    samples = t.rvs(df=dof, size=100000) * se + sm
    n_sim2[:,i] = samples
sims2 = pd.DataFrame(n_sim2)
sims2['max_ind'] = sims2.apply(np.argmax,axis=1)

In [14]:
probs2= sims2['max_ind'].value_counts()/len(n_sim2)

In [15]:
sims2['max_ind'].value_counts()

max_ind
4    99999
1        1
Name: count, dtype: int64

In [16]:
probs2

max_ind
4    0.99999
1    0.00001
Name: count, dtype: float64

In [17]:
vol['Probs']=probs2

In [18]:
vol.fillna(0)

Ad       Mean   Std Dev  DOF    Probs
0   1  42.076923  1.391324   53  0.00000
1   2  57.289474  2.822110   39  0.00001
2   3  41.803922  1.945134   52  0.00000
3   4  47.888889  2.017655   46  0.00000
4   5  90.640000  6.137285   26  0.99999

In [19]:
n_sim3 = np.multiply(n_sim,n_sim2)
sims3=pd.DataFrame(n_sim3)
sims3['max_ind']=sims3.apply(np.argmax,axis=1)

In [20]:
sims3['max_ind'] = sims3['max_ind']+1
probs3= sims3['max_ind'].value_counts()/len(sims3)

In [21]:
probs3= pd.DataFrame(probs3)
probs3['Ad'] = probs3.index

In [22]:
probs3

count  Ad
max_ind             
5        0.33845   5
2        0.20125   2
1        0.16850   1
4        0.15884   4
3        0.13296   3

In [23]:
df=vol.merge(probs3, how='inner', on='Ad')

In [24]:
df=df.fillna(0)

In [25]:
df=df.rename(columns={'Probs':'Conversion Volume Click Prob', 'max_ind':"Vols Exposure Prob"})

In [26]:
df.drop(columns=['Conversion Volume Click Prob'])

Ad       Mean   Std Dev  DOF    count
0   1  42.076923  1.391324   53  0.16850
1   2  57.289474  2.822110   39  0.20125
2   3  41.803922  1.945134   52  0.13296
3   4  47.888889  2.017655   46  0.15884
4   5  90.640000  6.137285   26  0.33845